In [388]:
import pathlib
import numpy as np
import pandas as pd
import json
import re
from tqdm import tqdm_notebook
import pdb
import spacy
import pytextrank
import datetime
import zipfile


In [340]:
h = pathlib.Path.home()
pet_data = open(f"{h}/downloads/PET 4.txt",'r').readlines()
ng_data = open(f"{h}/downloads/NG.txt",'r').readlines()
total_data = open(f"{h}/downloads/TOTAL.txt",'r').readlines()


In [224]:
pet_data_json = [json.loads(d) for d in pet_data if 'series_id' in d]
pet_categories_json = [json.loads(d) for d in pet_data if 'category_id' in d]

In [360]:
total_data_json = [json.loads(d) for d in total_data if 'series_id' in d]

In [289]:
ng_data_json = [json.loads(d) for d in ng_data if 'series_id' in d]
ng_categories_json = [json.loads(d) for d in ng_data if 'category_id' in d]

In [226]:
all_keys =[]
for d in pet_data_json:
    k = list(d.keys())
    all_keys = list(set(all_keys + k))
all_keys

['geography',
 'unitsshort',
 'units',
 'source',
 'iso3166',
 'start',
 'end',
 'data',
 'name',
 'series_id',
 'geography2',
 'description',
 'f',
 'last_updated',
 'copyright']

In [227]:
all_keys =[]
for d in pet_categories_json:
    k = list(d.keys())
    all_keys = list(set(all_keys + k))
all_keys

['category_id', 'notes', 'name', 'parent_category_id', 'childseries']

In [367]:
'yes' if re.search("c", "abcdef") else 'no'

'yes'

In [374]:
def petdata(d):
    if 'data' not in d:
        return None
    data_col = d['name'].replace(',','').replace('(','').replace(')','').lower()
    values = np.array(d['data'])
    dates = values[:,0].astype(int)
    try:
        vals = values[:,1].astype(float)
    except:
        vals = [np.nan if not re.search('^[-+]{0,1}[.0-9]{1,}$',str(v)) else float(v) for v in values[:,1]]#values[:,1].astype(float)
    freq = d['f']
    units = d['units'].lower()
    start = int(d['start'])
    end = int(d['end'])
    ret = {
        'name':data_col,
        'freq':freq,
        'units':units,
        'start':start,
        'end':end,
        'dates':dates,
        'vals':vals}
    return ret
#     return data_col,dates,vals


In [173]:
def normalize_date_array(darray,varray,all_dates):
    r = {ad:None for ad in all_dates}
    dv = {darray[i]:varray[i] for i in range(len(darray))}
    for k in dv.keys():
        r[k] = dv[k]
    
    ksorted = sorted(list(r.keys()),reverse=True)
    return [r[k] for k in ksorted]


In [293]:
def build_df_eia(json_data,freq='M'):
    tbeg = datetime.datetime.now()
    print(tbeg)
    freq_dict = {'M':'monthly','W':'weekly','D':'daily'}
    all_values = [petdata(v) for v  in json_data if v['f']==freq]
    most_dates = []
    for av in all_values:
        most_dates = list(set(most_dates + list(av['dates'])))
    all_dates = sorted(most_dates,reverse=True)
    cols = [av['name'] for av in all_values]
    darrays = [av['dates'] for av in all_values]
    varrays = [av['vals'] for av in all_values]
    vals = [
        normalize_date_array(darrays[i],varrays[i],all_dates)
        for i in range(len(cols))
    ]
    units = [av['units'] for av in all_values]
    freqs = [av['freq'] for av in all_values]
    df_cols = pd.DataFrame({'col':cols,'units':units,'freq':freqs})

    dict_df = {cols[i]:vals[i] for i in range(len(cols))}
    df_date = pd.DataFrame({'date':all_dates})
    
    df_vals = pd.DataFrame(dict_df)
    val_cols = list(df_vals.columns.values)
    df_vals['date'] = all_dates
    df_vals = df_vals[['date'] + val_cols]

    tend = datetime.datetime.now()
    print(tend)
    print(tend-tbeg)
    df_vals.columns = [c.lower() for c in df_vals.columns.values]
    return df_vals,df_cols


In [386]:
df_total_weekly,df_total_weekly_cols = build_df_eia(total_data_json,freq='W')
df_total_monthly,df_total_monthly_cols = build_df_eia(total_data_json,freq='M')
df_total_annual,df_total_annual_cols = build_df_eia(total_data_json,freq='A')


2020-08-30 08:40:35.021054
2020-08-30 08:40:35.024884
0:00:00.003830
2020-08-30 08:40:35.025525
2020-08-30 08:40:36.613802
0:00:01.588277
2020-08-30 08:40:36.634509
2020-08-30 08:40:36.937506
0:00:00.302997


In [294]:
df_ng_weekly,df_ng_weekly_cols = build_df_eia(ng_data_json,freq='W')

2020-08-29 16:07:24.125108
2020-08-29 16:07:24.167105
0:00:00.041997


In [295]:
df_ng_weekly

,date,weekly midwest region natural gas working underground storage weekly,weekly east region natural gas working underground storage weekly,weekly salt region natural gas working underground storage weekly,weekly nonsalt region natural gas working underground storage weekly,weekly south central region natural gas working underground storage weekly,weekly mountain region natural gas working underground storage weekly,weekly pacific region natural gas working underground storage weekly,weekly lower 48 states natural gas working underground storage weekly,henry hub natural gas spot price weekly,natural gas futures contract 1 weekly,natural gas futures contract 4 weekly,natural gas futures contract 2 weekly,natural gas futures contract 3 weekly
0,20200821,904.0,775.0,334.0,889.0,1223.0,212.0,306.0,3420.0,2.39,2.396,3.143,2.534,2.847
1,20200814,880.0,750.0,335.0,888.0,1223.0,209.0,313.0,3375.0,2.17,2.203,3.019,2.343,2.698
2,20200807,856.0,738.0,337.0,883.0,1219.0,206.0,314.0,3332.0,2.13,2.178,2.982,2.317,2.666
3,20200731,830.0,718.0,336.0,878.0,1214.0,202.0,311.0,3274.0,1.82,1.803,2.600,1.906,2.181
4,20200724,815.0,706.0,339.0,872.0,1211.0,196.0,313.0,3241.0,1.72,1.718,2.334,1.764,1.886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,19940121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.297,2.006,2.159,2.055
1388,19940114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.231,1.992,2.113,NaN
1389,19940107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.935,NaN,NaN
1390,19931231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.943,NaN,NaN


In [296]:
df_ng_weekly_cols

,col,units,freq
0,weekly midwest region natural gas working unde...,billion cubic feet,W
1,weekly east region natural gas working undergr...,billion cubic feet,W
2,weekly salt region natural gas working undergr...,billion cubic feet,W
3,weekly nonsalt region natural gas working unde...,billion cubic feet,W
4,weekly south central region natural gas worki...,billion cubic feet,W
5,weekly mountain region natural gas working und...,billion cubic feet,W
6,weekly pacific region natural gas working unde...,billion cubic feet,W
7,weekly lower 48 states natural gas working und...,billion cubic feet,W
8,henry hub natural gas spot price weekly,dollars per million btu,W
9,natural gas futures contract 1 weekly,dollars per million btu,W


In [297]:
df_ng_monthly,df_ng_monthly_cols = build_df_eia(ng_data_json,freq='M')

2020-08-29 16:18:59.937451
2020-08-29 16:19:01.391065
0:00:01.453614


In [298]:
df_ng_monthly

,date,natural gas futures contract 1 monthly,natural gas futures contract 3 monthly,henry hub natural gas spot price monthly,natural gas futures contract 4 monthly,natural gas futures contract 2 monthly,alaska natural gas underground storage volume monthly,federal offshore--gulf of mexico natural gas gross withdrawals from coalbed wells monthly,arkansas working natural gas underground storage capacity monthly,california working natural gas underground storage capacity monthly,...,u.s. natural gas exploratory wells drilled monthly,u.s. crude oil natural gas and dry developmental wells drilled monthly,u.s. crude oil natural gas and dry exploratory and developmental wells drilled monthly,u.s. crude oil exploratory wells drilled monthly,u.s. crude oil developmental wells drilled monthly,u.s. crude oil exploratory and developmental wells drilled monthly,u.s. crude oil natural gas and dry exploratory wells drilled monthly,price of kenai ak liquefied natural gas exports to china monthly,rhode island natural gas underground storage injections all operators monthly,kenai ak liquefied natural gas exports to china monthly
0,202007,1.77,1.97,1.77,2.40,1.83,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202006,1.70,1.84,1.63,1.97,1.78,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202005,1.81,2.08,1.75,2.14,2.00,38228.0,NaN,8894.0,373966.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202004,1.76,2.08,1.74,2.16,1.90,37179.0,NaN,8894.0,373966.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202003,1.73,1.87,1.79,1.97,1.78,37600.0,NaN,8894.0,373966.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,197305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,96.0,1697.0,2298.0,51.0,840.0,891.0,601.0,NaN,NaN,NaN
567,197304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,82.0,1406.0,1883.0,28.0,651.0,679.0,477.0,NaN,NaN,NaN
568,197303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.0,1598.0,2151.0,53.0,793.0,846.0,553.0,NaN,NaN,NaN
569,197302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61.0,1372.0,1896.0,52.0,643.0,695.0,524.0,NaN,NaN,NaN


In [299]:
df_pet_monthly,df_pet_monthly_cols = build_df_eia(pet_data_json,freq='M')


2020-08-29 16:19:14.679673
2020-08-29 16:20:01.049383
0:00:46.369710


In [301]:
df_pet_monthly

,date,north carolina gasoline midgrade all sales/deliveries by prime supplier monthly,south carolina gasoline midgrade all sales/deliveries by prime supplier monthly,virginia gasoline midgrade all sales/deliveries by prime supplier monthly,west virginia gasoline midgrade all sales/deliveries by prime supplier monthly,midwest padd 2 gasoline midgrade all sales/deliveries by prime supplier monthly,cushing ok wti spot price fob monthly,europe brent spot price fob monthly,north dakota gasoline midgrade all sales/deliveries by prime supplier monthly,ohio gasoline midgrade all sales/deliveries by prime supplier monthly,...,rocky mountain padd 4 net receipts by pipeline tanker and barge from other padds of lubricants monthly,rocky mountain padd 4 imports of kerosene monthly,refining district north louisiana-arkansas refinery and blender net production of isobutane monthly,iowa reformulated motor gasoline stocks at refineries bulk terminals and natural gas plants monthly,midwest padd 2 ending stocks of reformulated motor gasoline monthly,refining district appalachian no. 1 kerosene and light oils stocks at refineries monthly,kentucky reformulated motor gasoline stocks at refineries bulk terminals and natural gas plants monthly,gulf coast padd 3 imports of kerosene-type jet fuel bonded monthly,florida reformulated motor gasoline stocks at refineries bulk terminals and natural gas plants monthly,ohio reformulated motor gasoline stocks at refineries bulk terminals and natural gas plants monthly
0,202007,NaN,NaN,NaN,NaN,NaN,40.71,43.24,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202006,274.1,180.2,113.5,17.9,1186.3,38.31,40.27,20.0,129.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202005,255.5,163.1,92.6,13.5,982.1,28.56,29.38,19.1,104.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202004,200.0,115.3,65.2,10.2,786.6,16.55,18.38,14.7,91.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202003,244.6,155.6,98.9,13.3,1020.9,29.21,32.01,15.4,119.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,192005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1203,192004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1204,192003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1205,192002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [302]:
df_pet_monthly_cols

,col,units,freq
0,north carolina gasoline midgrade all sales/del...,thousand gallons per day,M
1,south carolina gasoline midgrade all sales/del...,thousand gallons per day,M
2,virginia gasoline midgrade all sales/deliverie...,thousand gallons per day,M
3,west virginia gasoline midgrade all sales/deli...,thousand gallons per day,M
4,midwest padd 2 gasoline midgrade all sales/del...,thousand gallons per day,M
...,...,...,...
79443,gulf coast padd 3 imports of kerosene-type jet...,thousand barrels,M
79444,gulf coast padd 3 imports of kerosene-type jet...,thousand barrels per day,M
79445,refining district appalachian no. 1 refinery a...,thousand barrels,M
79446,florida reformulated motor gasoline stocks at ...,thousand barrels,M


In [303]:
df_pet_weekly,df_pet_weekly_cols = build_df_eia(pet_data_json,freq='W')

2020-08-29 16:20:37.134096
2020-08-29 16:20:40.759938
0:00:03.625842


In [304]:
df_pet_weekly

,date,cushing ok crude oil future contract 4 weekly,u.s. imports of gasoline blending components weekly,u.s. ending stocks of gasoline blending components weekly,east coast padd 1 commercial crude oil imports excluding spr weekly,midwest padd 2 commercial crude oil imports excluding spr weekly,gulf coast padd 3 commercial crude oil imports excluding spr weekly,rocky mountain padd 4 commercial crude oil imports excluding spr weekly,west coast padd 5 commercial crude oil imports excluding spr weekly,u.s. commercial crude oil imports excluding spr weekly,...,gulf coast padd 3 ending stocks of reformulated rbob with ether gasoline blending components weekly,rocky mountain padd 4 rbob with ether gasoline blending components ending stocks weekly,west coast padd 5 ending stocks of reformulated rbob with ether gasoline blending components weekly,u.s. ending stocks of reformulated gtab gasoline blending components weekly,east coast padd 1 ending stocks of reformulated gtab gasoline blending components weekly,midwest padd 2 ending stocks of reformulated gtab gasoline blending components weekly,gulf coast padd 3 ending stocks of reformulated gtab gasoline blending components weekly,rocky mountain padd 4 ending stocks of reformulated gtab gasoline blending components weekly,west coast padd 5 ending stocks of reformulated gtab gasoline blending components weekly,u.s. ending stocks of reformulated rbob with alcohol gasoline blending components weekly
0,20200817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20200814,43.00,496.0,219590.0,667.0,2463.0,1387.0,413.0,801.0,5730.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20200810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20200807,42.51,878.0,223066.0,332.0,2739.0,1226.0,268.0,1056.0,5621.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20200803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681,19800201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3682,19800125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3683,19800118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3684,19800111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [305]:
df_pet_weekly_cols

,col,units,freq
0,cushing ok crude oil future contract 4 weekly,dollars per barrel,W
1,u.s. imports of gasoline blending components w...,thousand barrels per day,W
2,u.s. ending stocks of gasoline blending compon...,thousand barrels,W
3,east coast padd 1 commercial crude oil imports...,thousand barrels per day,W
4,midwest padd 2 commercial crude oil imports ex...,thousand barrels per day,W
...,...,...,...
1127,midwest padd 2 ending stocks of reformulated g...,thousand barrels,W
1128,gulf coast padd 3 ending stocks of reformulate...,thousand barrels,W
1129,rocky mountain padd 4 ending stocks of reformu...,thousand barrels,W
1130,west coast padd 5 ending stocks of reformulate...,thousand barrels,W


In [403]:
import base64
import io
import zipfile

def df_to_zipiofile(df,filename):
    sio2 = io.StringIO()
    df.to_csv(sio2,index=False)
    sio2.seek(0)
    zoio2 = io.BytesIO()
    f = zipfile.ZipFile(zoio2,'a',zipfile.ZIP_DEFLATED,False)
    f.writestr(filename,sio2.read())
    f.close() 
    zoio2.seek(0)
    return zoio2

def df_to_zipfile(df,filename,fullpath):
    zz = df_to_zipiofile(df,filename)
    ff = open(fullpath,'wb')
    ff.write(zz.getbuffer())
    ff.close()

In [405]:
df_to_zipfile(df_pet_monthly,'df_pet_monthly.csv','./temp_folder/df_pet_monthly.csv.zip')
df_to_zipfile(df_pet_weekly,'df_pet_weekly.csv','./temp_folder/df_pet_weekly.csv.zip')
df_to_zipfile(df_ng_monthly,'df_ng_monthly.csv','./temp_folder/df_ng_monthly.csv.zip')
df_to_zipfile(df_ng_weekly,'df_ng_weekly.csv','./temp_folder/df_ng_weekly.csv.zip')


## Begin NLP feature analysis here

In [333]:
from sklearn.feature_extraction.text import CountVectorizer


In [335]:
sentences = list(df_pet_weekly.columns.values)

In [339]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Learn vocabulary from sentences. 
vectorizer.fit(sentences)

# Get vocabularies.
vectorizer.vocabulary_

{'date': 54,
 'cushing': 52,
 'ok': 152,
 'crude': 51,
 'oil': 150,
 'future': 82,
 'contract': 48,
 'weekly': 219,
 'imports': 100,
 'gasoline': 84,
 'blending': 23,
 'components': 45,
 'ending': 66,
 'stocks': 196,
 'east': 62,
 'coast': 40,
 'padd': 158,
 'commercial': 44,
 'excluding': 72,
 'spr': 193,
 'midwest': 129,
 'gulf': 92,
 'rocky': 185,
 'mountain': 137,
 'west': 220,
 'exports': 73,
 'field': 74,
 'production': 169,
 'net': 140,
 'refiner': 174,
 'input': 103,
 'blender': 22,
 'distillate': 59,
 'fuel': 81,
 'greater': 88,
 '500': 7,
 'ppm': 164,
 'sulfur': 197,
 'new': 141,
 'england': 67,
 '1a': 1,
 'central': 36,
 'atlantic': 20,
 '1b': 2,
 'lower': 119,
 '1c': 3,
 'total': 204,
 'product': 168,
 'supplied': 198,
 'finished': 75,
 'motor': 136,
 'adjusted': 9,
 'gross': 89,
 'inputs': 104,
 'refineries': 175,
 'wti': 223,
 'spot': 192,
 'price': 166,
 'fob': 78,
 'europe': 71,
 'brent': 27,
 'reformulated': 177,
 'kerosene': 112,
 'type': 207,
 'jet': 109,
 'military'

In [320]:
def create_nlp_doc(df_lower):
    cols = df_lower.columns.values
    df_reports = pd.DataFrame({'report':cols[np.where(cols!='date')]})
    df_reports['colnum'] = df_reports.index.values

    reports_text = "\n".join(df_reports.report.values)[:999999]

    # example text
    text = reports_text
    # load a spaCy model, depending on language, scale, etc.
    nlp = spacy.load("en_core_web_sm")

    # add PyTextRank to the spaCy pipeline
    tr = pytextrank.TextRank()
    nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

    doc = nlp(text)

    # examine the top-ranked phrases in the document
    for p in doc._.phrases:
        print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
    return doc

In [321]:
df_data = df_pet_weekly.copy()
pet_doc  = create_nlp_doc(df_data)

0.1469    34  conventional retail gasoline prices
0.1446    87  retail gasoline prices
0.1440     2  reformulated retail gasoline prices
0.1415     5  regular conventional retail gasoline prices
0.1397     1  u.s. regular conventional retail gasoline prices
0.1389     3  regular reformulated retail gasoline prices
0.1380     1  u.s. premium conventional retail gasoline prices
0.1376     1  u.s. regular reformulated retail gasoline prices
0.1370     1  west coast regular conventional retail gasoline prices
0.1370     1  gulf coast regular conventional retail gasoline prices
0.1369     1  east coast regular conventional retail gasoline prices
0.1360     1  west coast midgrade conventional retail gasoline prices
0.1359     1  gulf coast midgrade conventional retail gasoline prices
0.1352     1  west coast premium conventional retail gasoline prices
0.1352     1  gulf coast premium conventional retail gasoline prices
0.1351     1  east coast premium conventional retail gasoline prices
0.13

In [332]:
df_docs = pd.DataFrame(
    {
        'prank':[p.rank for p in pet_doc._.phrases],
        'pcount':[p.count for p in pet_doc._.phrases],
        'ptext':[p.text for p in pet_doc._.phrases],
        'pcomb':[p.rank*p.count for p in pet_doc._.phrases]
    }
)
df_docs.sort_values('prank',ascending=False).head(50)

,prank,pcount,ptext,pcomb
0,0.146858,34,conventional retail gasoline prices,4.993172
1,0.144563,87,retail gasoline prices,12.576988
2,0.144038,2,reformulated retail gasoline prices,0.288076
3,0.141453,5,regular conventional retail gasoline prices,0.707265
4,0.139743,1,u.s. regular conventional retail gasoline prices,0.139743
5,0.138873,3,regular reformulated retail gasoline prices,0.416619
6,0.137989,1,u.s. premium conventional retail gasoline prices,0.137989
7,0.137640,1,u.s. regular reformulated retail gasoline prices,0.137640
8,0.137023,1,west coast regular conventional retail gasolin...,0.137023
9,0.136954,1,gulf coast regular conventional retail gasolin...,0.136954


## build column categories

In [843]:
PAD_REGEX = "(padd [0-9][a-z]{0,1}){1,2}"
PRODUCT_TYPE = "(kerosene-type jet fuel)|(gasoline)|(no 2[.]{0,1} diesel)|(propane( and propylene){0,1})|(crude)|(no. 2 heating oil)|(oxygenate)"
GRADE = "(regular)|(premium)|(midgrade)|(all grades)"
UNIT_TYPE = '(prices)|(inputs)|(net input)|(capacity)|(net production)|(production)|(imports)|(ending stocks)|(utilization)'
PROCESSOR_TYPE = '(refiner and blender)|(refiner)|(blender)|(oxygenate plant)|(gas plant)'
FORMULATION = "(all formulations)|(conventional)|(reformulated)"

def has_padd(col):
    return len(re.findall(PAD_REGEX,col.lower()))>0

def process_pad(cols,f,exclude_words=[]):
    vals = []
    for c in cols:
        if not has_padd(c):
            continue
        if any([ew in c for ew in exclude_words]):
            continue
        val = f(c)
        vals.append(val.strip().rstrip())
    return vals

def _padd(c):
    val = re.split(PAD_REGEX,c.lower())
    val = ''.join(val[1:-1])
    return val.strip().lstrip()

def get_padd(cols):
    return process_pad(cols,_padd)

def _padd_region(c):
    val = re.split(PAD_REGEX,c.lower())
    val = val[0].strip()
    g = GRADE
    val = re.split(g,val)[0]
    return val.strip().lstrip()

def get_padd_region(cols,**args):
    return process_pad(cols,_padd_region,**args)

def _pad_category(c):
    val = re.split(PAD_REGEX,c.lower())
    val = val[-1].strip().lower().split('of')[0]
    return val.strip().lstrip()

def get_padd_category(cols,**args):
    return process_pad(cols,_pad_category,**args)

def get_padd_category2(cols):
    vals = []
    for c in cols:
        if not has_padd(c):
            vals.append(None)
            continue
        val = re.split(PAD_REGEX,c.lower())
        val = val[-1].strip().lower().split('of')[0]
        if 'price' in val:
            vals.append(None)
            continue
        vals.append(val.strip().rstrip())
    return vals

def _pad_source(c):
    val = re.split(PAD_REGEX,c.lower())
    val = val[-1].strip().lower().split('of')[-1]
    return val.strip().lstrip()

def get_padd_source(cols,**args):
    return process_pad(cols,_pad_source,**args)


def get_padd_gasoline_type(cols,**args):
    vals = get_padd_source(cols,**args)
    vals = [re.split('gasoline',v)[0].strip().lstrip() for v in vals]
    return vals

def _padd_processor_type(c):
    val = re.split(PAD_REGEX,c.lower())
    val = ''.join(val[1:-1])
    return val.strip().lstrip()

def get_padd_processor_type(cols,**args):
    return process_pad(cols,_padd_processor_type,**args)

def get_ids(cols):
    def _findall(r,c):
        return ''.join([v for v in re.split(r,c.lower())[1:-1] if v is not None])
        l = re.findall(attribute_regexes[i],c)
        if l is None or len(l)<1:
            return ''
        if type(l[0]) == tuple:
            l = l[0]
        return ' '.join(l).strip().lstrip()
        
    all_attributes = {}
    attribute_regexes = [PAD_REGEX,PRODUCT_TYPE,GRADE,FORMULATION,UNIT_TYPE,PROCESSOR_TYPE]
    attribute_names = ['PAD_REGEX','PRODUCT_TYPE','GRADE','FORMULATION','UNIT_TYPE','PROCESSOR_TYPE']
    for i in range(len(attribute_regexes)):
        r = attribute_regexes[i]
        alist = [_findall(r,c) for c in cols]
        all_attributes[attribute_names[i]] = alist
    all_attributes['col'] = cols
    return pd.DataFrame(all_attributes)



In [786]:
set(get_padd_gasoline_type(df_pet_lower.columns.values))

{'',
 'all grades all formulations retail',
 'all grades conventional retail',
 'all grades reformulated retail',
 'commercial crude oil imports excluding spr weekly',
 'commercial kerosene-type jet fuel weekly',
 'conventional cbob',
 'conventional gtab',
 'conventional motor',
 'conventional other',
 'crude oil and petroleum products weekly',
 'crude oil weekly',
 'distillate fuel oil 0 to 15 ppm sulfur weekly',
 'distillate fuel oil greater than 15 to 500 ppm sulfur weekly',
 'distillate fuel oil greater than 2000 ppm sulfur weekly',
 'distillate fuel oil greater than 500 ppm sulfur weekly',
 'distillate fuel oil greater than 500 to 2000 ppm sulfur weekly',
 'distillate fuel oil weekly',
 'except california all grades all formulations retail',
 'except california all grades conventional retail',
 'except california all grades reformulated retail',
 'except california midgrade all formulations retail',
 'except california midgrade conventional retail',
 'except california midgrade re

In [838]:
df_cols = get_ids(df_pet_lower.columns.values)
df_cols

,PAD_REGEX,PRODUCT_TYPE,GRADE,FORMULATION,UNIT_TYPE,PROCESSOR_TYPE,col
0,,,,,,,date
1,,gasoline,midgrade,conventional,prices,,florida midgrade conventional retail gasoline prices weekly
2,,gasoline,premium,all formulations,prices,,boston ma premium all formulations retail gasoline prices weekly
3,,gasoline,premium,reformulated,prices,,san francisco ca premium reformulated retail gasoline prices weekly
4,,gasoline,premium,reformulated,prices,,new york harbor premium reformulated retail gasoline prices weekly
...,...,...,...,...,...,...,...
1128,padd 3,gasoline,,reformulated,ending stocks,,gulf coast padd 3 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
1129,padd 4,gasoline,,reformulated,ending stocks,,rocky mountain padd 4 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
1130,padd 5,gasoline,,reformulated,ending stocks,,west coast padd 5 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
1131,,gasoline,,reformulated,ending stocks,,u.s. ending stocks of reformulated rbob with ether gasoline blending components weekly


In [839]:
dfc = create_unique_values(df_cols)
dfc

,FORMULATION,GRADE,PAD_REGEX,PROCESSOR_TYPE,PRODUCT_TYPE,UNIT_TYPE
0,conventional,midgrade,padd 5,refiner blender and gas plant,gasoline,prices
1,all formulations,premium,padd 1c,refiner and blender,no 2 diesel,imports
2,reformulated,all grades,padd 1b,blender,propane,net production
3,,regular,padd 1a,refiner,propane and propylene and propylene,net input
4,,,padd 2,oxygenate plant,crude,ending stocks
5,,,padd 3,,oxygenate,production
6,,,padd 4,,no. 2 heating oil,utilization of refinery operable capacity
7,,,padd 1,,kerosene-type jet fuel,capacity
8,,,padd 4 and west coast padd 5,,gasoline non-oxygenate,inputs
9,,,padd 4 and padd 5,,,


In [824]:
padd_list = ['padd 1','padd 2','padd 3','padd 4','padd 5']
prod_type_list = ['gasoline']
grade_list = []#['regular']
unit_type_list = ['imports']#['ending stocks']
processor_type_list = []

all_true = pd.Series([True for _ in range(len(df_cols))])
do_screen = lambda c,l:all_true if len(l)<1 else df_cols[c].isin(l)

pad_regex_screen = do_screen('PAD_REGEX',padd_list)
pad_prod_type_screen = do_screen('PRODUCT_TYPE',prod_type_list)
pad_unit_type_screen = do_screen('UNIT_TYPE',unit_type_list)
pad_grade_screen = do_screen('GRADE',grade_list)
pad_processor_type_screen = do_screen('PROCESSOR_TYPE',processor_type_list)

screens = pad_regex_screen & pad_prod_type_screen & pad_unit_type_screen & pad_grade_screen & pad_processor_type_screen
cols_to_view = list(df_cols[screens].col.values)
cols_to_view = ['date'] + cols_to_view
df_screen = df_pet_lower[cols_to_view].fillna(0)
valid_cols = [c for c in df_screen.columns.values if df_screen[c].iloc[0:2].sum()>0]
df_screen[valid_cols]


,date,east coast padd 1 imports from all countries of motor gasoline blending components rbob weekly,west coast padd 5 imports of finished motor gasoline weekly,east coast padd 1 imports of finished motor gasoline weekly,east coast padd 1 imports of other conventional motor gasoline weekly,east coast padd 1 imports of conventional gtab gasoline blending components weekly,west coast padd 5 imports of conventional motor gasoline weekly,east coast padd 1 imports of conventional other gasoline blending components weekly,gulf coast padd 3 imports of conventional other gasoline blending components weekly,rocky mountain padd 4 conventional other gasoline blending components imports weekly,...,east coast padd 1 imports of total gasoline weekly,gulf coast padd 3 imports of total gasoline weekly,rocky mountain padd 4 imports of total gasoline weekly,west coast padd 5 imports of total gasoline weekly,east coast padd 1 imports of conventional cbob gasoline blending components weekly,east coast padd 1 imports of gasoline blending components weekly,gulf coast padd 3 imports of gasoline blending components weekly,rocky mountain padd 4 imports of gasoline blending components weekly,west coast padd 5 imports of gasoline blending components weekly,east coast padd 1 imports of reformulated rbob with alcohol gasoline blending components weekly
0,20200817,219.0,3.0,143.0,143.0,136.0,3.0,424.0,52.0,6.0,...,962.0,52.0,6.0,3.0,41.0,820.0,52.0,6.0,0.0,248.0
1,20200810,168.0,3.0,68.0,68.0,198.0,3.0,159.0,51.0,4.0,...,599.0,51.0,4.0,4.0,6.0,531.0,51.0,4.0,1.0,338.0
2,20200803,147.0,2.0,120.0,120.0,31.0,2.0,470.0,47.0,3.0,...,800.0,47.0,3.0,74.0,32.0,680.0,47.0,3.0,72.0,172.0
3,20200727,165.0,9.0,20.0,20.0,21.0,9.0,281.0,12.0,6.0,...,515.0,12.0,6.0,10.0,27.0,494.0,12.0,6.0,0.0,183.0
4,20200720,229.0,39.0,128.0,128.0,14.0,39.0,43.0,17.0,4.0,...,432.0,17.0,4.0,39.0,18.0,304.0,17.0,4.0,1.0,199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,20030623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
896,20030616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
897,20030609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
898,20030602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [811]:
pad_regex_screen
type(pad_unit_type_screen)

pandas.core.series.Series

In [795]:
from dashapp import dashapp2 as dashapp
from plotly.offline import iplot

ys_per_graph = 4
clen = len(df_screen.columns.values)
glen = int(clen/ys_per_graph) + (1 if clen % ys_per_graph > 0 else 0)
all_y_cols = [c for c in df_screen.columns.values if c!='date']
for i in range(glen):
    b = i*ys_per_graph
    e = b + ys_per_graph
    cols = ['date'] + all_y_cols[b:e]
    iplot(dashapp.plotly_plot(df_screen[cols],x_column='date'))

In [736]:
glen

0

In [710]:
set(get_padd(df_pet_lower.columns.values))

{'padd 1',
 'padd 1a',
 'padd 1b',
 'padd 1c',
 'padd 2',
 'padd 2r',
 'padd 3',
 'padd 4',
 'padd 4 and padd 5',
 'padd 4 and west coast padd 5',
 'padd 5'}

In [478]:
set(get_padd_region(df_pet_lower.columns.values))

{'',
 'central atlantic',
 'east coast',
 'gulf coast',
 'lower atlantic',
 'midwest',
 'new england',
 'rocky mountain',
 'rocky mountains',
 'weekly gulf coast',
 'weekly rocky mountain',
 'west coast'}

In [490]:
ss =set(get_padd_source(df_pet_lower.columns.values,exclude_words=['price']))
len(ss),ss

(54,
 {'commercial crude oil imports excluding spr weekly',
  'commercial kerosene-type jet fuel weekly',
  'conventional cbob gasoline blending components weekly',
  'conventional gtab gasoline blending components weekly',
  'conventional motor gasoline ed55 and lower weekly',
  'conventional motor gasoline greater than ed55 weekly',
  'conventional motor gasoline weekly',
  'conventional motor gasoline with fuel ethanol weekly',
  'conventional other gasoline blending components imports weekly',
  'conventional other gasoline blending components weekly',
  'crude oil and petroleum products weekly',
  'crude oil weekly',
  'distillate fuel oil 0 to 15 ppm sulfur weekly',
  'distillate fuel oil greater than 15 to 500 ppm sulfur weekly',
  'distillate fuel oil greater than 2000 ppm sulfur weekly',
  'distillate fuel oil greater than 500 ppm sulfur weekly',
  'distillate fuel oil greater than 500 to 2000 ppm sulfur weekly',
  'distillate fuel oil weekly',
  'finished conventional motor g

In [494]:
ss = set(get_padd_category(df_pet_lower.columns.values,exclude_words=['price']))
# ss = [s for s in ss if len(re.findall(unit_type,s))>0]#[s for s in ss if 'blender' in s]
len(ss),ss

(24,
 {'blender net production',
  'commercial crude oil imports excluding spr weekly',
  'conventional other gasoline blending components imports weekly',
  'efiner and blender net production',
  'ending stocks',
  'ending stocks excluding spr',
  'ending stocks excluding spr and including lease stock',
  'gross inputs into refineries weekly',
  'imports',
  'imports from',
  'imports from  all countries',
  'operable crude oil distillation capacity weekly',
  'oxygenate plant production',
  'percent utilization',
  'propane and propylene ending stocks excluding propylene at terminal weekly',
  'rbob with alcohol gasoline blending components imports weekly',
  'rbob with ether gasoline blending components ending stocks weekly',
  'rbob with ether gasoline blending components imports weekly',
  'refiner and blender net input',
  'refiner and blender net production',
  'refiner blender and gas plant net production',
  'refiner net input',
  'refiner net production',
  'reformulated gaso

In [501]:
ss = set(get_padd_gasoline_type(df_pet_lower.columns.values,exclude_words=['price']))
len(ss),ss

(42,
 {'',
  'commercial crude oil imports excluding spr weekly',
  'commercial kerosene-type jet fuel weekly',
  'conventional cbob',
  'conventional gtab',
  'conventional motor',
  'conventional other',
  'crude oil and petroleum products weekly',
  'crude oil weekly',
  'distillate fuel oil 0 to 15 ppm sulfur weekly',
  'distillate fuel oil greater than 15 to 500 ppm sulfur weekly',
  'distillate fuel oil greater than 2000 ppm sulfur weekly',
  'distillate fuel oil greater than 500 ppm sulfur weekly',
  'distillate fuel oil greater than 500 to 2000 ppm sulfur weekly',
  'distillate fuel oil weekly',
  'finished conventional motor',
  'finished motor',
  'finished reformulated motor',
  'fuel ethanol weekly',
  'gross inputs into refineries weekly',
  'kerosene-type jet fuel weekly',
  'military kerosene-type jet fuel weekly',
  'motor',
  'operable crude oil distillation capacity weekly',
  'other conventional motor',
  'other finished conventional motor',
  'other finished reformu

In [508]:
ss = set(get_padd_category(df_pet_lower.columns.values,exclude_words=['price']))
ss = [re.split(UNIT_TYPE,s)[0] for s in ss]
len(ss),set(ss)

(24,
 {'',
  'blender ',
  'commercial crude oil ',
  'conventional other gasoline blending components ',
  'efiner and blender ',
  'gross ',
  'operable crude oil distillation ',
  'oxygenate plant ',
  'percent ',
  'propane and propylene ',
  'rbob with alcohol gasoline blending components ',
  'rbob with ether gasoline blending components ',
  'refiner ',
  'refiner and blender ',
  'refiner blender and gas plant ',
  'reformulated gasoline non-oxygenated '})

In [511]:
ss = set([c for c in df_pet_lower.columns.values if len(re.findall(PROCESSOR_TYPE,c))>0])
len(ss),ss

(211,
 {'central atlantic padd 1b refiner blender and gas plant net production of propane and propylene weekly',
  'east coast padd 1  refiner and blender net production of commercial kerosene-type jet fuel weekly',
  'east coast padd 1  refiner and blender net production of distillate fuel oil greater than 500 ppm sulfur weekly',
  'east coast padd 1  refiner and blender net production of military kerosene-type jet fuel weekly',
  'east coast padd 1 blender net production of finished motor gasoline weekly',
  'east coast padd 1 gross inputs into refineries weekly',
  'east coast padd 1 oxygenate plant production of fuel ethanol weekly',
  'east coast padd 1 percent utilization of refinery operable capacity weekly',
  'east coast padd 1 refiner and blender net input of conventional cbob gasoline blending components weekly',
  'east coast padd 1 refiner and blender net input of conventional gtab gasoline blending components weekly',
  'east coast padd 1 refiner and blender net input of 